<a href="https://colab.research.google.com/github/9aditya9/makeuc-backend/blob/master/ml_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentencepiece

In [4]:

from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig

import gensim
from gensim.summarization import summarize


my_model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')


bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained(
    'facebook/bart-large-cnn')



ModuleNotFoundError: ignored

In [13]:
def summarize_textrank(text):
    print("input", text)

    # TextRank with Gensim
    summary = summarize(text, ratio=0.2)
    print("Summary:", summary)
    return summary


def summarize_t5(original_text):
    text = "summarize:" + original_text

    # encoding the input text
    input_ids = tokenizer.encode(text, return_tensors='pt', max_length=512)

    # Generating summary ids
    summary_ids = my_model.generate(input_ids)
    summary_ids
    
    # Decoding the tensor and printing the summary.
    t5_summary = tokenizer.decode(summary_ids[0])
    print(t5_summary)
    return t5_summary


def summarize_bart(original_text):
    # Encoding the inputs and passing them to model.generate()
    inputs = bart_tokenizer.batch_encode_plus(
        [original_text], return_tensors='pt')
    summary_ids = bart_model.generate(inputs['input_ids'], early_stopping=True)

    # Decoding and printing the summary
    bart_summary = bart_tokenizer.decode(
        summary_ids[0], skip_special_tokens=True)
    print("Summary:", bart_summary)
    return bart_summary

In [2]:
!pip install flask-ngrok flask-cors

In [3]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
from werkzeug.utils import secure_filename
import json

app = Flask(__name__)
run_with_ngrok(app)
CORS(app)

dict_f = {}

@app.route("/text", methods=["POST"])
def get_text():
  # data = request.args.get('data')
  data = request.get_json()
  # text = request.form.get['text']
  # print(text)
  print(data)
  dict_f[data['id']] = data['text']
  print(dict_f)
  dict_f[data['id']] = summarize_bart(data['text'])
  print(dict_f)
  resp = jsonify(success=True)
  return resp

@app.route("/summary")
def get_summary():
  q = request.args.get('id')
  print(dict_f[q])
  return jsonify({"summary": dict_f[q]})

@app.route("/")
def index():
  data = "Okay. Hi, I'm Ellie, and I only presenting our project Smartly Serve for Coastal Hacks today. A lot of people these days use the Internet and social media, and even though the other sometimes be a great place, me and my team and notice that after using the Internet, a lot of people might feel sad or angry or stressed without knowing what cause those negative emotions. If this continues for a long period of time, this might affect the person's mood drastically and affect very well being, or even lead to things such as anxiety or depression. Our team tried to propose a solution that solves this problem by making the users more self aware and self conscience of these feelings, and when they are triggered so that they can limit their usage or completely stop their usage of the websites that trigger their negative emotions a lot."
  # summary = summarize(data, ratio=0.1)
  summary = summarize_bart(data)
  print(summary)
  return f"<h3>{data}</h3><br><h4>{summary}</h4>"
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://64c1-35-233-155-68.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2021-10-10 12:21:09,713] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    r